In [14]:
import pysal as ps
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from libpysal.weights.contiguity import Queen
import libpysal
sns.set_style('white')


OSError: Could not load shared object file: llvmlite.dll

In [7]:
from libpysal.examples import load_example
elections = load_example('Elections')

OSError: Could not load shared object file: llvmlite.dll

In [8]:
elections.get_file_list()

NameError: name 'elections' is not defined

In [ ]:
#geoms = gpd.read_file(ps.examples.get_path('NAT.shp'))

In [ ]:
votes = gpd.read_file(elections.get_path('election.shp'))
%matplotlib inline
votes.plot()

In [ ]:
votes.head()

In [ ]:
print(votes['pct_dem_12'].mean())
print(votes['pct_dem_16'].mean())

In [ ]:
f,ax = plt.subplots(1,2, figsize=(2*3*1.6, 2))
for i,col in enumerate(['pct_dem_12','pct_dem_16']):
    sns.kdeplot(votes[col].values, shade=True, color='slategrey', ax=ax[i])
    ax[i].set_title(col.split('_')[1])

In [ ]:
votes.crs = {'init':'epsg:4326'}
votes = votes.to_crs(epsg='5070')

In [ ]:
f,ax = plt.subplots(2,2, figsize=(1.6*6 + 1,6*3), gridspec_kw=dict(width_ratios=(6,1)))
for i,col in enumerate(['pct_dem_12','pct_dem_16']):
    votes.plot(col, linewidth=.05, cmap='RdBu', ax=ax[i,0])
    ax[i,0].set_title(col.split('_')[1] + ' Two Party Vote (% Dem)')
    ax[i,0].set_xticklabels('')
    ax[i,0].set_yticklabels('')
    sns.kdeplot(votes[col].values, ax=ax[i,1], vertical=True, shade=True, color='slategrey')
    ax[i,1].set_xticklabels('')
    ax[i,1].set_ylim(0,1)
f.tight_layout()
plt.show()

In [ ]:
votes.dropna(subset=['pct_dem_12','pct_dem_16'], inplace=True)

In [ ]:
f,ax = plt.subplots(1,2, figsize=(4*2.1,4))
votes[['pct_dem_12','pct_dem_16']].plot.scatter('pct_dem_12','pct_dem_16', ax=ax[0])
ax[0].set_xlabel('2008 Two Party Vote (% Dem)')
ax[0].set_ylabel('2012 Two Party Vote (% Dem)')
ax[0].axis([0,1,0,1])
r = np.corrcoef(votes['pct_dem_12'].values, votes['pct_dem_16'].values)[0,1]

f.tight_layout()
plt.show()

In [ ]:
from splot.libpysal import plot_spatial_weights
w = Queen.from_dataframe(votes)
plot_spatial_weights(weights, votes)
plt.show()

In [ ]:
from splot.esda import moran_scatterplot
from splot.esda import plot_moran


# calculate Moran and plot
moran = Moran(votes['pct_dem_16'], w=W)
plot_moran(moran, zstandard=True, figsize=(10,4))
plt.show()
ax.set_xlabel('pct_dem_16')
ax.set_ylabel('Spatial Lag of pct_dem_16')
plt.show()

In [ ]:
?moran

## TASK: How can we isolate a p-value to validate whether our assumption of normality is true?